In [15]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Import the datetime module from the datetime library.
from citipy import citipy

from datetime import datetime 
import requests

# Import the API key.
from config import weather_api_key 
# Import linear regression from the SciPy stats module.
from scipy.stats import linregress 

In [20]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=35.782, size=100)
lngs = np.random.uniform(low=-180.000, high=180.000, size=100)
lat_lngs = zip(lats, lngs)
lat_lngs

In [21]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs) 

In [22]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities) 

67

In [23]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key 

In [24]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1 

Beginning Data Retrieval     
-----------------------------


In [25]:
# Loop through all the cities in our list.
for i in range(len(cities)):

    # Group cities in sets of 50 for logging purposes.
    if (i % 5 == 0 and i >= 5):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]  

In [26]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 5 == 0 and i >= 5):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          }) 
        

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

# Indicate that Data Loading is complete.

Processing Record 1 of Set 14 | hobart
Processing Record 2 of Set 14 | catamarca
Processing Record 3 of Set 14 | khorramshahr
Processing Record 4 of Set 14 | kavaratti
Processing Record 5 of Set 14 | caravelas
Processing Record 1 of Set 15 | punta arenas
Processing Record 2 of Set 15 | ushuaia
Processing Record 3 of Set 15 | busselton
Processing Record 4 of Set 15 | necochea
Processing Record 5 of Set 15 | albany
Processing Record 1 of Set 16 | avarua
Processing Record 2 of Set 16 | tsihombe
City not found. Skipping...
Processing Record 3 of Set 16 | jamestown
Processing Record 4 of Set 16 | kruisfontein
Processing Record 5 of Set 16 | new norfolk
Processing Record 1 of Set 17 | puerto ayora
Processing Record 2 of Set 17 | san patricio
Processing Record 3 of Set 17 | carnarvon
Processing Record 4 of Set 17 | flinders
Processing Record 5 of Set 17 | rio grande
Processing Record 1 of Set 18 | bambous virieux
Processing Record 2 of Set 18 | east london
Processing Record 3 of Set 18 | taol

In [27]:
city_data

[{'City': 'Hobart', 'Lat': -42.8794, 'Lng': 147.3294},
 {'City': 'Catamarca', 'Lat': -28.4696, 'Lng': -65.7852},
 {'City': 'Khorramshahr', 'Lat': 30.4397, 'Lng': 48.1664},
 {'City': 'Kavaratti', 'Lat': 10.5669, 'Lng': 72.642},
 {'City': 'Caravelas', 'Lat': -17.7125, 'Lng': -39.2481},
 {'City': 'Punta Arenas', 'Lat': -53.15, 'Lng': -70.9167},
 {'City': 'Ushuaia', 'Lat': -54.8, 'Lng': -68.3},
 {'City': 'Busselton', 'Lat': -33.65, 'Lng': 115.3333},
 {'City': 'Necochea', 'Lat': -38.5473, 'Lng': -58.7368},
 {'City': 'Albany', 'Lat': 42.6001, 'Lng': -73.9662},
 {'City': 'Avarua', 'Lat': -21.2078, 'Lng': -159.775},
 {'City': 'Jamestown', 'Lat': 42.097, 'Lng': -79.2353},
 {'City': 'Kruisfontein', 'Lat': -34.0033, 'Lng': 24.7314},
 {'City': 'New Norfolk', 'Lat': -42.7826, 'Lng': 147.0587},
 {'City': 'Puerto Ayora', 'Lat': -0.7393, 'Lng': -90.3518},
 {'City': 'San Patricio', 'Lat': 28.017, 'Lng': -97.5169},
 {'City': 'Carnarvon', 'Lat': -24.8667, 'Lng': 113.6333},
 {'City': 'Flinders', 'Lat': -3

In [30]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10) 

,City,Lat,Lng
0,Hobart,-42.8794,147.3294
1,Catamarca,-28.4696,-65.7852
2,Khorramshahr,30.4397,48.1664
3,Kavaratti,10.5669,72.6420
4,Caravelas,-17.7125,-39.2481
5,Punta Arenas,-53.1500,-70.9167
6,Ushuaia,-54.8000,-68.3000
7,Busselton,-33.6500,115.3333
8,Necochea,-38.5473,-58.7368
9,Albany,42.6001,-73.9662


In [32]:
city_data_df.to_csv(r'lat_lng.csv', encoding='utf-8', index=False) 